In [8]:
import pandas as pd
from IPython.display import display

In [9]:
df = pd.read_csv("../data/raw/clinical_data.csv")

In [10]:
pd.set_option('display.max_columns', None)


# Descripcion de las variables

## 1. Variables Demográficas y Socioeconómicas
Estas definen quién es el paciente y su entorno.

<h3>Age recode with "<"1 year olds and 90+ :</h3> Edad agrupada. Es preferible a la edad continua para evitar identificar personas muy ancianas. Úsala para ver la distribución por cohortes de edad.

<h3>Race and origin (recommended by SEER) :</h3> Clasificación detallada de etnia (incluye específicos como filipino, japonés, etc.).

<h3>Race recode (White, Black, Other) :</h3> Versión simplificada de la anterior. Ideal para análisis de disparidades si los grupos pequeños no tienen suficiente volumen.

<h3>Origin recode NHIA (Hispanic, Non-Hisp) :</h3> Específico para identificar población de origen hispano.
El algoritmo NHIA (NAACCR Hispanic Identification Algorithm) es el estándar de oro para identificar la etnia hispana en bases de datos de EE. UU., combinando el apellido, el lugar de nacimiento y la autodeclaración.

Spanish-Hispanic-Latino: El paciente se identifica como hispano.

Non-Spanish-Hispanic-Latino: El paciente no es de origen hispano.

<h3>Median household income inflation adj to 2023 :</h3> Ingreso promedio del área de residencia del paciente. Es una variable "proxy" del nivel socioeconómico (Determinantes Sociales de la Salud).

<h3>Rural-Urban Continuum Code :</h3> Clasifica el condado del paciente desde metrópolis grandes hasta zonas rurales remotas. Vital para analizar acceso a salud.Esta variable clasifica el lugar de residencia del paciente. Es vital para tu modelo porque la supervivencia suele ser mayor en zonas con grandes centros hospitalarios.

Counties in metropolitan areas ge 1 million pop: Ciudades grandes (Nueva York, Los Ángeles, etc.). Máximo acceso a tecnología.

Counties in metropolitan areas of 250,000 to 1 million: Áreas urbanas medianas.

Nonmetropolitan counties (adjacent or not): Zonas rurales. Aquí la supervivencia puede ser menor debido a la distancia que el paciente debe recorrer para recibir quimioterapia o cirugía.

Unknown/no match: Generalmente registros con códigos postales incompletos o de territorios fuera de la zona estándar.

<h3>PRCDA 2020 (PurchasedReferred Care Delivery Area)</h3>
Esta variable es un indicador de geografía médica en Estados Unidos.

PRCDA: El paciente reside en un condado donde el Servicio de Salud Indígena (IHS) puede financiar atención médica externa.

Not PRCDA: El condado no pertenece a esta categoría.

Unknown PRCDA: Información geográfica incompleta.



## 2. Variables del Tumor (Biología y Localización)
Definen qué tipo de cáncer es.

<h3>Primary Site - labeled :</h3> Indica en qué parte específica del pulmón se originó el tumor.

C34.1, C34.2, C34.3: Lóbulos superior, medio e inferior. Es vital para la supervivencia; los tumores en el lóbulo superior a veces tienen pronósticos distintos por la cercanía a grandes vasos.

C34.0 (Main bronchus): El bronquio principal. Suele ser más central y difícil de operar.

C34.9 (Lung, NOS): "Not Otherwise Specified". Significa que en la historia clínica solo dice "pulmón" sin aclarar el lóbulo.

C34.8 (Overlapping lesion): El tumor es tan grande que abarca más de un lóbulo.


<h3>Histologic Type ICD-O-3 :</h3> El código de morfología. Crucial para distinguir entre Cáncer de pulmón de células no pequeñas (Adenocarcinoma, Escamoso) y Células pequeñas.

<h3>Grade Clinical (2018+) :</h3> Qué tan diferenciadas (anormales) están las células.Representa qué tan "anormales" se ven las células bajo el microscopio.

1, 2, 3, 4: Grado de diferenciación (1: Muy parecido al tejido normal/lento; 4: Indiferenciado/muy agresivo).

A, B, C, D: Son códigos nuevos introducidos en 2018 para tipos específicos de tumores (ej. Bien diferenciado, Moderadamente, etc.).

9 / Blank(s): Información no disponible o grado desconocido.

<h3>Total number of in situ/malignant tumors for patient :</h3> Indica si el paciente ha tenido otros cánceres. Un número alto puede complicar el pronóstico.

## 3. Estadificación (Severidad y Extensión)

Esta es la parte más técnica. SEER usa diferentes sistemas según el año de diagnóstico.

<h3>Year of diagnosis :</h3> Año en que se confirmó el diagnóstico.

<h3>Derived AJCC T (7th ed, 2010-2015) :</h3> 

Describe el tamaño del tumor original y si ha invadido tejidos cercanos dentro del pulmón o estructuras adyacentes.

T1a, T1b, T2a, T2b, T3, T4: Representan el crecimiento en centímetros y la invasión (T1 es pequeño, T4 es invasión de órganos vitales).

T1NOS / T2NOS: "Not Otherwise Specified". Sabemos que es T1 o T2, pero el informe no aclara si es 'a' o 'b'.
    T1 (T1a, T1b): Tumor pequeño (≤ 3 cm) rodeado por pulmón o pleura visceral.

    T2 (T2a, T2b): Tumor más grande (> 3 cm pero ≤ 7 cm) o que invade el bronquio principal o la pleura visceral.

TX: El tumor primario no puede ser evaluado.

T0: No hay evidencia de tumor primario (muy raro en cáncer de pulmón diagnosticado).

T0: No hay evidencia de tumor primario.


<h3>Derived AJCC N, 7th ed (2010-2015) :</h3> Describe si el cáncer se ha propagado a los ganglios linfáticos regionales y cuáles están afectados.

N0: El cáncer no ha llegado a los ganglios. Es el mejor escenario para una cirugía.

N1: Afectación de ganglios cercanos al pulmón afectado.

N2: El cáncer llegó al centro del pecho (mediastino) del mismo lado. Aquí la cirugía es mucho más compleja.

N3: Afectación del lado opuesto o arriba de la clavícula. Generalmente se considera inoperable.

NX: No se pudo evaluar (a veces por falta de pruebas).

Blank(s)/nan: Datos fuera del periodo 2010-2015.

<h3>Derived AJCC M, 7th ed (2010-2015) :</h3> Describe si el cáncer ha viajado a órganos lejanos.

M0: No hay evidencia de metástasis a distancia.

M1a: El cáncer está en ambos pulmones o en el líquido que rodea al pulmón/corazón.

M1b: Metástasis en órganos lejanos (cerebro, huesos, hígado). Es la fase más grave.

M1NOS: Se sabe que hay metástasis, pero el reporte no especifica de qué tipo.

<h3>Tumor Size Summary (2016+) / CS tumor size (2004-2015) :</h3> El tamaño exacto del tumor en milímetros. Se usan columnas distintas dependiendo del año de diagnóstico.

<h3>Combined Summary Stage (2004+) :</h3> Esta es la de gravedad.

In situ: El cáncer no ha atravesado la capa basal (no es invasivo aún).

Localized only: Solo está en el pulmón.

Regional (Direct extension / Node / Both): Se ha extendido a tejidos vecinos o a los ganglios linfáticos cercanos.

Distant site(s): Metástasis. El cáncer viajó a otros órganos (Etapa IV).

<h3>CS extension (2004-2015) :</h3>  Los números representan qué tan profundo "creció" el tumor.

100: El tumor está confinado al pulmón (localizado).

300 - 400: El tumor ha llegado a la pleura (la capa que recubre el pulmón) o a la pared del pecho.

700 - 800: El tumor invade estructuras críticas como el corazón, el esófago o la tráquea.

999: Información desconocida o no documentada.

Blank(s): Pacientes diagnosticados fuera del rango 2004-2015.

<h3>Derived EOD 2018 Stage Group :</h3> La estadificación más reciente para casos actuales, Este es el sistema más moderno (basado en la 8ª Edición de la AJCC). Es mucho más granular que las versiones anteriores.

1A1, 1A2, 1A3, 1B: Etapas muy tempranas. La diferencia entre 1A1 y 1A2 son milímetros de tamaño.

2A, 2B: Etapas intermedias.

3A, 3B, 3C: Etapas avanzadas regionales (el cáncer está en los ganglios del centro del pecho).

4A, 4B: Metástasis. 4B indica que el cáncer se ha extendido a múltiples órganos lejanos.

Códigos Especiales:

0: Carcinoma in situ (no invasivo).

OC: Cáncer Oculto (se ven células cancerosas en el esputo, pero no encuentran el tumor en imágenes).

88: No aplicable.

99: Desconocido.

<h3>AJCC ID (2018+) :</h3>
Esta variable es puramente administrativa pero vital para limpiar tus datos.

Lung: Confirma que el caso fue evaluado con el protocolo de pulmón.

No AJCC Chapter / Soft tissue sarcoma...: ¡Cuidado aquí! En tu EDA notarás que algunos tumores en el pulmón no son "cáncer de pulmón" típico (carcinoma), sino sarcomas (tejido blando).

Importancia para ML: Te sugiero filtrar tu dataset para quedarte solo con los que dicen Lung si quieres un modelo especializado en los tipos más comunes.

## 4. Tratamiento y Fuente de Datos

<h3>RX Summ--Surg Prim Site (1998+) :</h3> Indica si se realizó cirugía y qué tipo (ej. lobectomía, neumonectomía).

<h3>Reason no cancer-directed surgery :</h3> Por qué no se operó Esta variable es fundamental si quieres entender por qué un paciente con un tumor operable no llegó a quirófano.

Surgery performed: El paciente recibió el tratamiento estándar.

Not recommended: El equipo médico decidió que la cirugía no era la mejor opción (comúnmente porque el cáncer ya está en etapa IV/Metastásica).

Recommended but not performed, patient refused: Un factor psicológico o social. El paciente decidió no operarse a pesar del consejo médico.

Not recommended, contraindicated due to other cond: El paciente tenía otras enfermedades (corazón, riñones, etc.) que hacían que la anestesia o la cirugía fueran más peligrosas que el propio cáncer.

Not performed, patient died prior to recommended surgery: Casos de progresión extremadamente rápida.

Unknown; death certificate; or autopsy only: Casos donde la información es de baja calidad porque el cáncer se descubrió después de la muerte.

<h3>Type of Reporting Source :</h3> De dónde viene el dato (Hospital, laboratorio, certificado de defunción). Ayuda a evaluar la calidad del dato.
Indica quién envió el dato a SEER. Esto afecta directamente la fiabilidad de tu modelo de ML.

Hospital inpatient/outpatient or clinic: Es el dato más fiable. Viene de un historial clínico completo.

Laboratory only: El dato viene de una biopsia. Puede que falte información sobre si el paciente sobrevivió o no a largo plazo.

Death certificate only (DCO): ¡Atención! En ML, estos casos suelen eliminarse. Si el registro viene solo del certificado de defunción, los Survival months suelen ser 0, lo que puede "engañar" a tu modelo SVC haciéndole creer que todos mueren instantáneamente.

Nursing/convalescent home/hospice: Pacientes que ya estaban en cuidados paliativos o residencias de ancianos.

<h3>Diagnostic Confirmation :</h3> Cómo se confirmó el cáncer (Biopsia microscópica, solo imagenología o solo clínica).

Positive histology: Confirmado con biopsia de tejido (Máxima certeza).

Positive exfoliative cytology: Confirmado por células en fluidos (esputo, líquido pleural).

Radiography / Clinical diagnosis only: No hubo biopsia; se determinó por tomografía o síntomas. Esto suele ocurrir en pacientes muy graves que no aguantarían una biopsia.

Direct visualization: El cirujano lo vio con sus propios ojos durante una operación pero no se analizó el tejido.

## 5. Resultados (Supervivencia y Estado Final)

<h3>Survival months :</h3> Tiempo transcurrido desde el diagnóstico hasta la muerte o el último seguimiento.

<h3>Vital status recode (study cutoff used) :</h3> 
Alive / Dead: Es una variable binaria simple. Indica si el paciente estaba vivo en el último contacto registrado por el hospita

<h3>SEER cause-specific death classification :</h3> Dead (attributable to this cancer dx): El paciente murió a causa del cáncer de pulmón.

Alive or dead of other cause: El paciente sigue vivo O murió por otra cosa (ej. un accidente o un infarto).

Nota técnica: En medicina, esto se llama "censura". Si alguien muere de un infarto, no podemos decir que el tratamiento del cáncer falló, simplemente "salieron" del estudio por otra razón.

Dead (missing/unknown COD): Murió, pero no se sabe por qué. Para un modelo de alta precisión, estos registros suelen descartarse

<h3>Sequence number :</h3> Indica el orden de este tumor en la vida del paciente (si es el primero, el segundo, etc.).

One primary only: Este es el único cáncer que el paciente ha tenido en su vida (según el registro). Representa la mayoría de los casos.

1st of 2 or more primaries: Es el primer cáncer detectado en un paciente que luego desarrolló otros.

2nd, 3rd... of N or more: Indica que el paciente ya tuvo otros cánceres antes que este.

In [11]:
df["Rural-Urban Continuum Code"].value_counts()


Rural-Urban Continuum Code
Counties in metropolitan areas ge 1 million pop                 291753
Counties in metropolitan areas of 250,000 to 1 million pop      113711
Nonmetropolitan counties adjacent to a metropolitan area         49586
Counties in metropolitan areas of lt 250 thousand pop            46491
Nonmetropolitan counties not adjacent to a metropolitan area     35039
Unknown/missing/no match (Alaska or Hawaii - Entire State)         821
Unknown/missing/no match/Not 1990-2023                              17
Name: count, dtype: int64

In [19]:
df=df[~df["Rural-Urban Continuum Code"].isin(["Unknown/missing/no match (Alaska or Hawaii - Entire State)", "Unknown/missing/no match/Not 1990-2023"])]

In [20]:
df["Rural-Urban Continuum Code"].value_counts()


Rural-Urban Continuum Code
Counties in metropolitan areas ge 1 million pop                 291753
Counties in metropolitan areas of 250,000 to 1 million pop      113711
Nonmetropolitan counties adjacent to a metropolitan area         49586
Counties in metropolitan areas of lt 250 thousand pop            46491
Nonmetropolitan counties not adjacent to a metropolitan area     35039
Name: count, dtype: int64

In [21]:
df

KeyboardInterrupt: 

# Analisis de variables

In [15]:
df.head()

,Age recode with <1 year olds and 90+,Race and origin (recommended by SEER),"Race recode (White, Black, Other)",Primary Site - labeled,Histologic Type ICD-O-3,Grade Clinical (2018+),Diagnostic Confirmation,Combined Summary Stage with Expanded Regional Codes (2004+),"Derived AJCC T, 7th ed (2010-2015)","Derived AJCC N, 7th ed (2010-2015)","Derived AJCC M, 7th ed (2010-2015)",Tumor Size Summary (2016+),AJCC ID (2018+),CS tumor size (2004-2015),Survival months,Vital status recode (study cutoff used),SEER cause-specific death classification,Year of diagnosis,Median household income inflation adj to 2023,RX Summ--Surg Prim Site (1998+),Reason no cancer-directed surgery,Rural-Urban Continuum Code,Type of Reporting Source,PRCDA 2020,Total number of in situ/malignant tumors for patient,CS extension (2004-2015),Derived EOD 2018 Stage Group Recode (2018+),Sequence number,"Origin recode NHIA (Hispanic, Non-Hisp)"
0,65-69 years,All races/ethnicities,White,"C34.1-Upper lobe, lung",8140,2,Positive histology,Regional by both direct extension and lymph no...,Blank(s),Blank(s),Blank(s),070,Lung,Blank(s),0059,Alive,Alive or dead of other cause,2018,"$120,000+",33,Surgery performed,Counties in metropolitan areas ge 1 million pop,Hospital inpatient/outpatient or clinic,Not PRCDA,2,Blank(s),3A,2nd of 2 or more primaries,Non-Spanish-Hispanic-Latino
1,65-69 years,All races/ethnicities,White,C34.0-Main bronchus,8140,Blank(s),"Positive exfoliative cytology, no positive his...",Distant site(s)/node(s) involved,Blank(s),Blank(s),Blank(s),109,Blank(s),Blank(s),0027,Dead,Dead (attributable to this cancer dx),2016,"$110,000 - $119,999",0,Not recommended,Counties in metropolitan areas ge 1 million pop,Hospital inpatient/outpatient or clinic,Not PRCDA,2,Blank(s),Blank(s),2nd of 2 or more primaries,Non-Spanish-Hispanic-Latino
2,65-69 years,All races/ethnicities,Black,"C34.9-Lung, NOS",8041,Blank(s),Positive histology,Unknown/unstaged/unspecified/DCO,TX,NX,M0,Blank(s),Blank(s),999,0000,Dead,Dead (attributable to this cancer dx),2014,"$110,000 - $119,999",0,Not recommended,Counties in metropolitan areas ge 1 million pop,Physicians office/private medical practitioner...,Not PRCDA,2,999,Blank(s),2nd of 2 or more primaries,Non-Spanish-Hispanic-Latino
3,80-84 years,All races/ethnicities,White,"C34.1-Upper lobe, lung",8550,Blank(s),Positive histology,Regional by direct extension only,T3,N0,M0,Blank(s),Blank(s),028,0052,Dead,Dead (attributable to this cancer dx),2015,"$100,000 - $109,999",33,Surgery performed,Counties in metropolitan areas ge 1 million pop,Hospital inpatient/outpatient or clinic,Not PRCDA,2,100,Blank(s),2nd of 2 or more primaries,Non-Spanish-Hispanic-Latino
4,65-69 years,All races/ethnicities,White,"C34.1-Upper lobe, lung",8140,3,Positive histology,Localized only,Blank(s),Blank(s),Blank(s),033,Lung,Blank(s),0012,Dead,Alive or dead of other cause,2020,"$120,000+",0,Not recommended,Counties in metropolitan areas ge 1 million pop,Hospital inpatient/outpatient or clinic,Not PRCDA,3,Blank(s),1B,2nd of 2 or more primaries,Non-Spanish-Hispanic-Latino


In [16]:
# Creamos una tabla resumen de las 29 columnas
resumen_columnas = pd.DataFrame({
    'Tipo': df.dtypes,
    'No Nulos': df.count(),
    'Nulos': df.isnull().sum()
})

# Forzamos a Pandas a mostrar todas las filas de este resumen
pd.set_option('display.max_rows', None)

display(resumen_columnas)

,Tipo,No Nulos,Nulos
Age recode with <1 year olds and 90+,object,537418,0
Race and origin (recommended by SEER),object,537418,0
"Race recode (White, Black, Other)",object,537418,0
Primary Site - labeled,object,537418,0
Histologic Type ICD-O-3,int64,537418,0
Grade Clinical (2018+),object,537418,0
Diagnostic Confirmation,object,537418,0
Combined Summary Stage with Expanded Regional Codes (2004+),object,537418,0
"Derived AJCC T, 7th ed (2010-2015)",object,536987,431
"Derived AJCC N, 7th ed (2010-2015)",object,536987,431


In [17]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Histologic Type ICD-O-3,537418.0,8122.758679,126.531359,8000.0,8041.0,8072.0,8140.0,9581.0
Year of diagnosis,537418.0,2016.977267,3.142155,2012.0,2014.0,2017.0,2020.0,2022.0
RX Summ--Surg Prim Site (1998+),537418.0,9.517683,20.696553,0.0,0.0,0.0,0.0,99.0
Total number of in situ/malignant tumors for patient,537418.0,1.422438,0.751866,1.0,1.0,1.0,2.0,34.0


In [18]:
#eliminacion de datos duplicados
df.drop_duplicates()

KeyboardInterrupt: 

In [ ]:
df.nunique()

Age recode with <1 year olds and 90+                            20
Race and origin (recommended by SEER)                            1
Race recode (White, Black, Other)                                4
Primary Site - labeled                                           6
Histologic Type ICD-O-3                                        192
Grade Clinical (2018+)                                          11
Diagnostic Confirmation                                          9
Combined Summary Stage with Expanded Regional Codes (2004+)      7
Derived AJCC T, 7th ed (2010-2015)                              11
Derived AJCC N, 7th ed (2010-2015)                               6
Derived AJCC M, 7th ed (2010-2015)                               5
Tumor Size Summary (2016+)                                     301
AJCC ID (2018+)                                                  5
CS tumor size (2004-2015)                                      292
Survival months                                               

In [ ]:
# 1. Identificar columnas no numéricas
columnas_no_num = df.select_dtypes(exclude=['number']).columns

# 2. Iterar y mostrar usando f-strings
for col in columnas_no_num:
    print(col)
    print(df[col].unique())
   

Age recode with <1 year olds and 90+
['65-69 years' '80-84 years' '85-89 years' '55-59 years' '60-64 years'
 '90+ years' '75-79 years' '70-74 years' '40-44 years' '50-54 years'
 '45-49 years' '35-39 years' '30-34 years' '20-24 years' '25-29 years'
 '00 years' '10-14 years' '01-04 years' '15-19 years' '05-09 years']
Race and origin (recommended by SEER)
['All races/ethnicities']
Race recode (White, Black, Other)
['White' 'Black'
 'Other (American Indian/AK Native, Asian/Pacific Islander)' 'Unknown']
Primary Site - labeled
['C34.1-Upper lobe, lung' 'C34.0-Main bronchus' 'C34.9-Lung, NOS'
 'C34.3-Lower lobe, lung' 'C34.2-Middle lobe, lung'
 'C34.8-Overlapping lesion of lung']
Grade Clinical (2018+)
['2' 'Blank(s)' '3' '9' '1' '4' 'D' 'C' 'B' 'H' 'A']
Diagnostic Confirmation
['Positive histology'
 'Positive exfoliative cytology, no positive histology' 'Unknown'
 'Radiography without microscopic confirm' 'Clinical diagnosis only'
 'Direct visualization without microscopic confirmation'
 'Po